In [1]:
import pandas as pd
from collections import defaultdict
from stop import Stop

## Chargement des données

In [2]:
routes = pd.read_csv('IDFM-gtfs/routes.txt')
trips = pd.read_csv('IDFM-gtfs/trips.txt')
stop_times = pd.read_csv('IDFM-gtfs/stop_times.txt')
stops = pd.read_csv('IDFM-gtfs/stops.txt')

/var/folders/0y/kz93z2gx1fn6s9_d1wp7pq300000gn/T/ipykernel_57078/1799971113.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv('IDFM-gtfs/trips.txt')


In [3]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order
0,IDFM:C01371,IDFM:Operator_100,1,1,NaN,1,NaN,FFBE00,000000,NaN
1,IDFM:C01286,IDFM:Operator_100,322,322,NaN,3,NaN,82C8E6,000000,NaN
2,IDFM:C01153,IDFM:Operator_100,124,124,NaN,3,NaN,FF82B4,000000,NaN
3,IDFM:C00173,IDFM:1081,18,18,NaN,3,NaN,DB006B,FFFFFF,NaN
4,IDFM:C00391,IDFM:1058,4334,4334,NaN,3,NaN,E2001A,FFFFFF,NaN


## Filtrage des données

### Lignes et parcours

In [4]:
# Sélection des lignes et parcours de metro uniquement
metro_routes = routes[routes['route_type'] == 1]['route_id']
metro_trips = trips[trips['route_id'].isin(metro_routes)]

In [5]:
metro_trips.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
49891,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49892,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_3,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49894,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49900,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_5,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49901,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_7,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0


In [6]:
# On sélectionne les 16 premiers parcours de chaque ligne afin d'inclure tout les chemins possibles
selected_trips = metro_trips.groupby(['route_id', 'direction_id']).head(16).reset_index()
#selected_trips = metro_trips.drop_duplicates(subset=['route_id', 'direction_id', 'shape_id'])

selected_trips.head()

,index,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,49891,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
1,49892,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_3,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
2,49894,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
3,49900,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_5,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
4,49901,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_7,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0


### Arrêts

In [7]:
stop_times = stop_times[stop_times['trip_id'].isin(selected_trips['trip_id'])]
stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])
stop_times.count()

trip_id           10857
arrival_time      10857
departure_time    10857
stop_id           10857
stop_sequence     10857
pickup_type       10857
drop_off_type     10857
local_zone_id         0
stop_headsign         0
timepoint         10857
dtype: int64

In [8]:
def to_seconds(t):
    h, m, s = map(int, t.split(":"))
    return h * 3600 + m * 60 + s

# Expression des heures en secondes
stop_times['arr_sec'] = stop_times['arrival_time'].apply(to_seconds)
stop_times['dep_sec'] = stop_times['departure_time'].apply(to_seconds)

In [9]:
stops_metro = stops[stops['stop_id'].isin(stop_times['stop_id'])]
stops_metro.count()

stop_id                752
stop_code                0
stop_name              752
stop_desc                0
stop_lon               752
stop_lat               752
zone_id                752
stop_url                 0
location_type          752
parent_station         752
stop_timezone          752
level_id                 0
wheelchair_boarding    752
platform_code            0
dtype: int64

In [10]:
stops_commercial = stops_metro[stops_metro['location_type'] == 1][['stop_id', 'stop_name']]
parent_dict = stops_commercial.set_index('stop_id')['stop_name'].to_dict()

In [11]:
physical_stops = stops_metro[stops_metro["location_type"] == 0]
#physical_stops = stops[
 #   (stops['stop_id'].isin(stop_times['stop_id'])) &
  #  (stops['location_type'] == 0)
#]
physical_stops.count()

stop_id                752
stop_code                0
stop_name              752
stop_desc                0
stop_lon               752
stop_lat               752
zone_id                752
stop_url                 0
location_type          752
parent_station         752
stop_timezone          752
level_id                 0
wheelchair_boarding    752
platform_code            0
dtype: int64

In [12]:
stop_dict = {} # {stop_id: Stop()}

for row in physical_stops.itertuples(index=False):
    name = parent_dict.get(row.parent_station, row.stop_name)  # nom de la station commerciale si dispo
    key = row.parent_station if pd.notna(row.parent_station) else row.stop_id
    if key not in stop_dict:
        stop_dict[row.stop_id] = Stop(key, name, row.stop_lat, row.stop_lon)

In [13]:
stop_times = stop_times.merge(physical_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on='stop_id', how='left')
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint,arr_sec,dep_sec,stop_name,stop_lat,stop_lon
0,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:30:00,07:30:00,IDFM:22175,0,0,1,NaN,NaN,1,27000,27000,Corvisart,48.829860,2.350611
1,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:31:00,07:31:00,IDFM:22173,1,0,0,NaN,NaN,1,27060,27060,Glacière,48.831170,2.343497
2,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:32:00,07:32:00,IDFM:22160,2,0,0,NaN,NaN,1,27120,27120,Saint-Jacques,48.832916,2.337154
3,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:34:00,07:34:00,IDFM:22158,3,0,0,NaN,NaN,1,27240,27240,Denfert-Rochereau,48.834313,2.332872
4,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:35:00,07:35:00,IDFM:22155,4,0,0,NaN,NaN,1,27300,27300,Raspail,48.839085,2.330550


In [14]:
stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])
stop_times['next_stop_id'] = stop_times.groupby('trip_id')['stop_id'].shift(-1)
stop_times['next_arr_sec'] = stop_times.groupby('trip_id')['arr_sec'].shift(-1)
stop_times['duration'] = stop_times['next_arr_sec'] - stop_times['dep_sec']

## Construction du graphe

In [15]:
edges = stop_times.dropna(subset=['next_stop_id']).copy()
edges['duration'] = edges['next_arr_sec'] - edges['dep_sec']

In [16]:
edges['u'] = edges['stop_id'].map(stop_dict)
edges['v'] = edges['next_stop_id'].map(stop_dict)
edges.head()
edges.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint,arr_sec,dep_sec,stop_name,stop_lat,stop_lon,next_stop_id,next_arr_sec,duration,u,v
0,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:30:00,07:30:00,IDFM:22175,0,0,1,NaN,NaN,1,27000,27000,Corvisart,48.829860,2.350611,IDFM:22173,27060.0,60.0,Corvisart (IDFM:71013),Glacière (IDFM:71026)
1,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:31:00,07:31:00,IDFM:22173,1,0,0,NaN,NaN,1,27060,27060,Glacière,48.831170,2.343497,IDFM:22160,27120.0,60.0,Glacière (IDFM:71026),Saint-Jacques (IDFM:71041)
2,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:32:00,07:32:00,IDFM:22160,2,0,0,NaN,NaN,1,27120,27120,Saint-Jacques,48.832916,2.337154,IDFM:22158,27240.0,120.0,Saint-Jacques (IDFM:71041),Denfert-Rochereau (IDFM:71056)
3,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:34:00,07:34:00,IDFM:22158,3,0,0,NaN,NaN,1,27240,27240,Denfert-Rochereau,48.834313,2.332872,IDFM:22155,27300.0,60.0,Denfert-Rochereau (IDFM:71056),Raspail (IDFM:71088)
4,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:35:00,07:35:00,IDFM:22155,4,0,0,NaN,NaN,1,27300,27300,Raspail,48.839085,2.330550,IDFM:22172,27360.0,60.0,Raspail (IDFM:71088),Edgar Quinet (IDFM:71108)


In [17]:
graph = defaultdict(dict)
for row in edges.itertuples(index=False):
    u, v, w = row.u, row.v, row.duration
    if pd.notnull(u) and pd.notnull(v):
        if v not in graph[u] or graph[u][v] > w:
            graph[u][v] = w
            graph[v][u] = w
        

### Tests du graphe

In [18]:
def is_connected(graph):
    if not graph:
        return True  # An empty graph is trivially connected

    visited = set()
    start = next(iter(graph))  # Start from any node
    
    def dfs(node):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor)

    dfs(start)
    return (len(visited) == len(graph), visited)


In [19]:
def test_connexite(graph):
    est_connnexe, visited = is_connected(graph)
    if not est_connnexe:
        for g in graph:
            if g not in visited:
                print(g)
    else:
        print('Le graphe est connexe.')

test_connexite(graph)

Le graphe est connexe.


In [20]:
len(graph)

308

In [21]:
name_to_stops = defaultdict(list)
for stop in stop_dict.values():
    name_to_stops[stop.name].append(stop)

### Test classe MetroGraph

In [22]:
from MetroGraph import MetroGraph

In [23]:
g = MetroGraph(stop_dict, edges)

In [24]:
odeon_stops = g.get_stops_by_name("Châtelet")
for stop in odeon_stops:
    print(stop, "→", g.voisins(stop))

Châtelet (IDFM:71264) → {Pont Neuf (IDFM:71253): 60.0, Pont Marie (Cité des Arts) (IDFM:71217): 60.0, Louvre - Rivoli (IDFM:73630): 60.0, Hôtel de Ville (IDFM:415852): 60.0, Gare de Lyon (IDFM:73626): 180.0, Pyramides (IDFM:73788): 60.0, Les Halles (IDFM:73794): 60.0, Cité (IDFM:71223): 60.0}


In [25]:
est_connexe, visited = g.is_connected()
print("Graphe connexe :", est_connexe)

Graphe connexe : True


## Sauvegarde du graphe

In [26]:
import pickle

with open("metro_graph.pkl", "wb") as f:
    pickle.dump(g, f)